In [1]:
import pandas as pd
import os
import pickle
import bmra_prep
import bmra_prep.pathway_activity.prediction

In [2]:
data_dir = "/home/jing/Phd_project/project_GBM/gbm_Scripts/gbm_Scripts_bmra_u251mg/01_outputs_2020/"
#info_dir = data_dir

out_dir = "02_outputs_2020"

os.makedirs(out_dir, exist_ok = True)


# Load Data

In [3]:
# load metdadata dict and extract used elements
with open(os.path.join(data_dir, "metadata.pickle"), "rb") as f:
    all_metadata = pickle.load(f)

n_modules = all_metadata["n_modules"]
n_genes = all_metadata["n_genes"]
n_experiments = all_metadata["n_experiments"]

modules = all_metadata["modules"]
exp_ids = all_metadata["exp_ids"]
genes = all_metadata["genes"]

In [4]:
# load data
L1000_df = pd.read_csv(
    os.path.join(data_dir, "L1000_Data_norm_data.csv"),
    index_col = 0,
)

x = L1000_df.values
x.shape

(978, 98)

In [5]:
# load doses and perturbation matrix
inhib_conc_matrix = pd.read_csv(
    os.path.join(data_dir, "inhib_conc_annotated.csv"),
    index_col = 0,
).values

ic50_matrix = pd.read_csv(
    os.path.join(data_dir, "ic50_annotated.csv"),
    index_col = 0,
).values

# gamma_matrix = pd.read_csv(
#     os.path.join(data_dir, "gamma_annotated.csv"),
#     index_col = 0,
# ).values

pert_matrix = pd.read_csv(
    os.path.join(data_dir, "pert_annotated.csv"),
    index_col = 0,
).values

In [6]:
# y_true = (1 + gamma_matrix * inhib_conc_matrix / ic50_matrix) / (1 + inhib_conc_matrix / ic50_matrix)

y_true = 1 / (1 + inhib_conc_matrix / ic50_matrix)

display(y_true.shape)
y_true

(10, 98)

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.000000

## Run models

In [7]:
a_coeffs = bmra_prep.pathway_activity.prediction.predict_coeffs(
    x, y_true, pert_matrix, 200_000, 10, 10, 10, 100)

In [8]:
a_coeffs_df = pd.DataFrame(a_coeffs, index = modules, columns = genes)
a_coeffs_df.to_csv(os.path.join(out_dir, "a_coeffs.csv"))
#a_coeffs_df = pd.read_csv(os.path.join(out_dir,'a_coeffs.csv'),index_col=0)
#a_coeffs = a_coeffs_df.values
display(a_coeffs_df.astype(bool).sum(axis='columns'))
display(a_coeffs_df)

CDK1        978
CDK2        978
CDK4_6      978
PI3K        978
ERK         978
PDGFR       978
Aurora      978
Estrogen    978
EGFR        978
p53         978
dtype: int64

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
CDK1,-1.573487e-05,1.027982e-05,0.000005,-0.000012,0.000010,-0.000003,-1.874870e-05,0.000002,-0.000011,4.052895e-07,...,0.000004,-6.299181e-06,1.732925e-05,-1.893409e-05,2.466935e-05,-2.178078e-07,0.000008,0.000034,1.648823e-05,0.000028
CDK2,1.323886e-05,-2.096552e-06,0.000026,-0.000002,-0.000015,0.000019,6.975697e-06,-0.000020,0.000017,-8.113540e-06,...,-0.000022,-6.269276e-06,1.352240e-05,-9.664288e-06,-5.711190e-07,-7.451189e-06,-0.000002,0.000017,-2.239589e-07,0.000001
CDK4_6,1.070209e-05,-3.548823e-05,0.000005,-0.000028,0.000011,0.000008,-7.641625e-07,-0.000006,0.000038,3.110050e-05,...,-0.000024,1.089219e-05,9.286324e-07,7.751989e-07,1.514168e-05,1.263788e-05,0.000002,-0.000012,-1.269377e-06,-0.000007
PI3K,1.859332e-05,-5.548878e-06,0.000017,0.000003,0.000016,0.000026,5.609847e-05,-0.000030,0.000011,1.052333e-05,...,0.000001,-1.388171e-05,-2.098980e-06,2.334252e-05,-1.403945e-05,1.052335e-05,-0.000013,-0.000030,3.716149e-05,0.000008
ERK,-1.824550e-05,2.021571e-05,0.000009,0.000022,-0.000007,-0.000012,1.463043e-06,0.000001,-0.000003,-1.028439e-05,...,-0.000033,1.020667e-05,-8.722903e-07,1.536902e-05,1.565360e-05,1.780949e-01,0.000002,-0.000022,7.173283e-06,-0.000015
PDGFR,6.341077e-06,-8.211143e-07,-0.000015,-0.000013,0.000008,0.000019,1.545059e-05,-0.000018,0.000003,4.506390e-06,...,0.000007,7.093305e-07,1.062136e-06,-1.155660e-05,2.928041e-05,2.292333e-02,0.000020,-0.000031,-1.537829e-06,-0.000004
Aurora,1.075273e-05,-2.324454e-05,0.000006,0.000005,-0.000009,0.000010,-9.514199e-06,-0.000002,0.000013,-2.070377e-05,...,-0.000002,2.002902e-06,-7.498273e-06,1.019878e-05,-6.502675e-06,2.068021e-05,0.000008,0.000011,-4.330254e-05,0.000011
Estrogen,1.356297e-05,-2.072255e-05,-0.000035,0.000009,-0.000007,-0.000002,-7.069984e-06,-0.000011,-0.000005,1.739621e-05,...,0.000006,-3.748551e-05,-3.497697e-06,1.215388e-06,-8.364951e-06,1.409922e-03,0.000009,-0.000017,3.242990e-05,0.000013
EGFR,-1.144378e-07,3.502205e-06,-0.000009,0.000002,-0.000014,0.000002,9.766408e-06,-0.000013,0.174642,3.038326e-06,...,0.000017,-6.643234e-06,3.136597e-06,2.489725e-05,-2.247707e-06,3.112109e-05,0.000009,0.000035,-6.024301e-06,-0.000007
p53,-2.106851e-05,1.067172e-05,0.000011,0.000013,0.000014,0.000005,-1.509402e-05,0.000034,0.000011,-7.165281e-06,...,-0.000013,5.654010e-06,-5.160891e-06,7.588846e-06,-2.098266e-05,2.842252e-06,0.000011,-0.000023,-1.977311e-06,-0.000008


In [9]:
#pathway_activity = a_coeffs @ x
#pathway_activity.shape

In [12]:
R_global = bmra_prep.pathway_activity.calc_global_response_from_pathway_activity(
    bmra_prep.pathway_activity.calc_pathway_activity(x,a_coeffs),
    modules,
    L1000_df.columns
)
R_global_df = R_global.dataframe
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:A19,ASG002_U251MG_24H:A20,ASG002_U251MG_24H:A21,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,...,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:O13,ASG002_U251MG_24H:O14,ASG002_U251MG_24H:O15,ASG002_U251MG_24H:O22,ASG002_U251MG_24H:O23,ASG002_U251MG_24H:O24,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK1,0.081089,0.125011,0.173547,-0.607866,0.265283,-0.891511,-0.757245,-0.798125,-0.029274,-0.941224,...,0.286276,0.153923,0.215892,0.297630,0.178274,-0.388498,0.151805,-0.470206,0.187746,-0.481100
CDK2,0.106075,0.116774,0.009522,0.098108,0.053207,0.060171,0.086467,0.065324,0.093244,0.064641,...,0.008067,0.014290,0.020957,0.036406,0.027402,-0.001307,0.047737,0.050421,0.094933,-0.162768
CDK4_6,-0.392868,-0.012786,0.138232,-0.308891,0.001852,-0.599087,-0.385783,-0.248940,-0.058611,-1.644640,...,0.338615,-0.103578,-0.122352,0.450784,-0.048856,0.068505,-0.013369,-1.817373,-1.729637,-1.096661
PI3K,-1.727518,-1.936152,-1.574534,-1.770995,-1.363744,-0.462367,-0.097555,-0.133710,-0.363719,-0.480663,...,-0.550722,-1.123907,0.487393,-1.039486,-1.445170,-1.430944,-0.861013,-1.064608,-1.146680,-0.804919
ERK,-0.056669,-0.902130,-0.573469,-1.376985,-0.240381,-0.070063,-0.050381,0.030621,0.073343,0.244829,...,-0.526101,-1.984386,-1.648063,-1.602062,-1.648365,-1.860993,-2.000578,-0.567394,-1.981519,-0.280683
PDGFR,-1.675321,-0.122161,-0.609842,-0.101268,-0.548778,-0.670138,0.034691,0.067000,0.120590,0.164171,...,-1.116301,-1.519194,-0.416419,-1.324917,-0.532642,-0.051473,-0.709364,-1.862738,-1.005205,-0.338867
Aurora,0.389183,-0.869003,0.133138,0.536756,0.582836,-2.000814,-1.327355,-1.641404,-1.581067,-1.072098,...,0.155092,-0.999820,-0.461707,0.320468,-0.201117,0.633963,0.636174,-0.907239,-1.061106,-0.521568
Estrogen,0.178734,-0.059205,-0.459808,0.059799,-1.570045,-0.187582,-0.071305,0.115753,0.110214,0.153080,...,-1.393176,-0.309838,-0.315023,-0.316760,-0.626627,-0.100765,-1.549463,-0.857565,-0.178473,0.786974
EGFR,-0.246361,-1.112098,0.877428,-1.993334,-0.578342,-1.030612,-0.845636,-1.106294,-1.329879,-1.927600,...,1.008659,-1.167423,0.955037,-0.911178,-1.181477,0.985696,-0.739315,-0.681555,-0.716416,-0.303777
p53,-0.248589,0.238370,0.088570,0.116936,-0.312133,-1.426714,-1.595636,-1.572175,-1.512323,-0.779571,...,0.423060,0.179602,-0.000437,0.424994,0.064967,0.406100,0.383319,0.149433,0.034424,-0.005261


In [13]:
R_global_df.to_csv(os.path.join(out_dir, "R_global_annotated.csv"))
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:A19,ASG002_U251MG_24H:A20,ASG002_U251MG_24H:A21,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,...,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:O13,ASG002_U251MG_24H:O14,ASG002_U251MG_24H:O15,ASG002_U251MG_24H:O22,ASG002_U251MG_24H:O23,ASG002_U251MG_24H:O24,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK1,0.081089,0.125011,0.173547,-0.607866,0.265283,-0.891511,-0.757245,-0.798125,-0.029274,-0.941224,...,0.286276,0.153923,0.215892,0.297630,0.178274,-0.388498,0.151805,-0.470206,0.187746,-0.481100
CDK2,0.106075,0.116774,0.009522,0.098108,0.053207,0.060171,0.086467,0.065324,0.093244,0.064641,...,0.008067,0.014290,0.020957,0.036406,0.027402,-0.001307,0.047737,0.050421,0.094933,-0.162768
CDK4_6,-0.392868,-0.012786,0.138232,-0.308891,0.001852,-0.599087,-0.385783,-0.248940,-0.058611,-1.644640,...,0.338615,-0.103578,-0.122352,0.450784,-0.048856,0.068505,-0.013369,-1.817373,-1.729637,-1.096661
PI3K,-1.727518,-1.936152,-1.574534,-1.770995,-1.363744,-0.462367,-0.097555,-0.133710,-0.363719,-0.480663,...,-0.550722,-1.123907,0.487393,-1.039486,-1.445170,-1.430944,-0.861013,-1.064608,-1.146680,-0.804919
ERK,-0.056669,-0.902130,-0.573469,-1.376985,-0.240381,-0.070063,-0.050381,0.030621,0.073343,0.244829,...,-0.526101,-1.984386,-1.648063,-1.602062,-1.648365,-1.860993,-2.000578,-0.567394,-1.981519,-0.280683
PDGFR,-1.675321,-0.122161,-0.609842,-0.101268,-0.548778,-0.670138,0.034691,0.067000,0.120590,0.164171,...,-1.116301,-1.519194,-0.416419,-1.324917,-0.532642,-0.051473,-0.709364,-1.862738,-1.005205,-0.338867
Aurora,0.389183,-0.869003,0.133138,0.536756,0.582836,-2.000814,-1.327355,-1.641404,-1.581067,-1.072098,...,0.155092,-0.999820,-0.461707,0.320468,-0.201117,0.633963,0.636174,-0.907239,-1.061106,-0.521568
Estrogen,0.178734,-0.059205,-0.459808,0.059799,-1.570045,-0.187582,-0.071305,0.115753,0.110214,0.153080,...,-1.393176,-0.309838,-0.315023,-0.316760,-0.626627,-0.100765,-1.549463,-0.857565,-0.178473,0.786974
EGFR,-0.246361,-1.112098,0.877428,-1.993334,-0.578342,-1.030612,-0.845636,-1.106294,-1.329879,-1.927600,...,1.008659,-1.167423,0.955037,-0.911178,-1.181477,0.985696,-0.739315,-0.681555,-0.716416,-0.303777
p53,-0.248589,0.238370,0.088570,0.116936,-0.312133,-1.426714,-1.595636,-1.572175,-1.512323,-0.779571,...,0.423060,0.179602,-0.000437,0.424994,0.064967,0.406100,0.383319,0.149433,0.034424,-0.005261
